#### Import Libraries

In [ ]:
import sempy
import sempy.fabric as fabric
import sempy_labs as labs

from pyspark.sql.functions import col, count
import pandas as pd

from sempy_labs import admin, graph
from sempy_labs.tom import connect_semantic_model

from datetime import datetime
import msal, requests

from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

import requests, time, pandas as pd, concurrent.futures as cf

#### Global Variables

In [ ]:
archive_workspace = 'your-archive-workspace-id'

KEY_VAULT_URI ="your-keyvault-url"
TENANT_ID_SECRET = "TenantID"
CLIENT_ID_SECRET  = "ClientID"
CLIENT_SECRET_SECRET = "ClientSecret"

SCOPE = ["https://api.fabric.microsoft.com/.default"]

#### Get secrets from Keyvault

In [ ]:
class _FabricKVToken:
    def get_token(self, *scopes, **kwargs):
        t = mssparkutils.credentials.getToken("https://vault.azure.net")
        return type("Tok", (), {"token": t, "expires_on": int(time.time()) + 3600})()

# ---------  Read SP credentials from Key Vault ----------
kv = SecretClient(vault_url=KEY_VAULT_URI, credential=_FabricKVToken())
TENANT_ID    = kv.get_secret(TENANT_ID_SECRET).value
CLIENT_ID    = kv.get_secret(CLIENT_ID_SECRET).value
CLIENT_SECRET = kv.get_secret(CLIENT_SECRET_SECRET).value

#### Get Unused Reports

In [ ]:
reports = spark.sql("""
SELECT r.`Report Id`
FROM LH_Monitoring.dbo.dim_reports r
LEFT JOIN LH_Monitoring.dbo.dim_workspaces w 
    ON r.`Workspace Id` = w.Id
LEFT JOIN LH_Monitoring.dbo.fact_audit_logs l 
    ON r.`Report Id` = l.`Report Id`
   AND l.Operation = 'ViewReport'
   AND l.`Creation Time` > date_sub(current_timestamp(), 90)
WHERE w.Type = 'Workspace'
  AND r.`Created Date` < date_sub(current_timestamp(), 90)
  AND r.IsDeleted = 0
GROUP BY r.`Report Id`
HAVING COUNT(l.Id) = 0
""")

report_list = reports.rdd.flatMap(lambda x: x).collect()


#### Copy Unused Reports to Archive Workspace

In [ ]:
for r in report_list:

    # Set required variables
    report_name = spark.sql(f"SELECT `Report Name` FROM LH_Monitoring.dbo.dim_reports WHERE `Report Id` = '{r}'").collect()[0][0]
    date_suffix = datetime.now().strftime("%d%m%Y%H%M%S")
    destination_report_name = report_name+"-"+date_suffix
    report_workspace = spark.sql(f"SELECT `Workspace Id` FROM LH_Monitoring.dbo.dim_reports WHERE `Report Id` = '{r}'").collect()[0][0]

    # Copy report
    df = labs.copy_item(item= r
    , type= 'Report'
    , source_workspace= report_workspace
    , target_name= destination_report_name
    ,target_workspace= archive_workspace
    ,overwrite=True
    ,keep_existing_bindings= True
    )

    # Insert record into Report Archive History Table
    spark.sql(f"INSERT INTO report_archive_history VALUES ('{r}', '{report_workspace}', now(), '{report_name}', '{destination_report_name}',NULL, NULL)")

#### Delete Report from Source Workspace

In [ ]:
for r in report_list:
    REPORT_ID     = r   # the report to delete
    WORKSPACE_ID  = spark.sql(f"SELECT `Workspace Id` FROM LH_Monitoring.dbo.dim_reports WHERE `Report Id` = '{r}'").collect()[0][0]

    # Get token for Fabric REST
    authority = f"https://login.microsoftonline.com/{TENANT_ID}"
    app = msal.ConfidentialClientApplication(CLIENT_ID, authority=authority, client_credential=CLIENT_SECRET)
    token = app.acquire_token_for_client(scopes=SCOPE)
    access_token = token["access_token"]

    # Call Delete Report
    url = f"https://api.fabric.microsoft.com/v1/workspaces/{WORKSPACE_ID}/reports/{REPORT_ID}"
    resp = requests.delete(url, headers={"Authorization": f"Bearer {access_token}"})
    print(resp.status_code, resp.text or "Deleted (200)")